### Exploratory Data Analysis: Healthcare Data from Maven Analytics

This detailed EDA focuses on patient-level data sourced from Maven Analytics, representing approximately 900 patients from a healthcare institute in Boston. It's important to note that the records are synthetic and do not reflect actual patient data. The primary objectives of this EDA include understanding the data structure of the table, transforming it into more PowerBI-friendly flat files, and gaining insights into what the data represents. Key aspects of the analysis include basic counts, data integrity checks, and distinguishing between Oncology and Non-Oncology data segments where applicable.

The analysis will cover:
- Data structure overview and transformation for PowerBI compatibility.
- Basic statistical summaries and distributions.
- Sanity checks to ensure data reliability.
- Segmentation and analysis between Oncology and Non-Oncology patient groups.

This EDA aims to provide a comprehensive understanding of the dataset's characteristics and insights into healthcare trends within the Boston healthcare institute context.

In [2]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)

In [3]:
# Read the CSV files to understand the data structure of the flat files, starting with the data dictionary

data_dict = pd.read_csv("data_dictionary.csv")
encounters = pd.read_csv("encounters.csv")
organizations = pd.read_csv("organizations.csv")
patients = pd.read_csv("patients.csv")
payers = pd.read_csv("payers.csv")
procedures = pd.read_csv("procedures.csv")

### Understand the data structure of each table
Also explore some fields

In [5]:
# Data Dictionary
data_dict.head(2)

,Table,Field,Description
0,patients,Id,Primary Key. Unique Identifier of the patient.
1,patients,BIRTHDATE,The date (YYYY-MM-DD) the patient was born.


In [6]:
# These are the names of the tables we will work with 
set(data_dict.Table.to_list())

{'encounters', 'organizations', 'patients', 'payers', 'procedures'}

##### Encounters Table
Records containing each de-identified patient visit to the facility

In [8]:
encounters.head(2)

,Id,START,STOP,CLAIM_YEAR,CLAIM_MONTH,PATIENT,ORGANIZATION,PAYER,ENCOUNTERCLASS,CODE,DESCRIPTION,BASE_ENCOUNTER_COST,TOTAL_CLAIM_COST,PAYER_COVERAGE,REASONCODE,REASONDESCRIPTION
0,32c84703-2481-49cd-d571-3899d5820253,02-01-2011,02-01-2011,2011,1,3de74169-7f67-9304-91d4-757e0f3a14d2,d78e84ec-30aa-3bba-a33a-f29a3a454662,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,ambulatory,185347001,Encounter for problem (procedure),85.55,1018.02,0.0,NaN,NaN
1,c98059da-320a-c0a6-fced-c8815f3e3f39,03-01-2011,03-01-2011,2011,1,d9ec2e44-32e9-9148-179a-1653348cc4e2,d78e84ec-30aa-3bba-a33a-f29a3a454662,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,outpatient,308335008,Patient encounter procedure,142.58,2619.36,0.0,NaN,NaN


In [9]:
print(f"Shape of the encounters table: {encounters.shape}")
print(f"Columns in the encounter table: {encounters.columns}")
print(f"Number of Unique Patients: {encounters.PATIENT.nunique()}\n-------------")

Shape of the encounters table: (27891, 16)
Columns in the encounter table: Index(['Id', 'START', 'STOP', 'CLAIM_YEAR', 'CLAIM_MONTH', 'PATIENT',
       'ORGANIZATION', 'PAYER', 'ENCOUNTERCLASS', 'CODE', 'DESCRIPTION',
       'BASE_ENCOUNTER_COST', 'TOTAL_CLAIM_COST', 'PAYER_COVERAGE',
       'REASONCODE', 'REASONDESCRIPTION'],
      dtype='object')
Number of Unique Patients: 974
-------------


In [10]:
# Basic Data Points to note
print(f"Number of Unique Encounters: {encounters.Id.nunique()}")
print(f"Since this is the same as number of rows, this means no duplicates are found in the Encounter IDs \n-------------")
print(f"Unique values of Encounters Class: {encounters.ENCOUNTERCLASS.unique()}\n-------------")
print(f"Example values of Diagnoses: {encounters.REASONDESCRIPTION.unique()[1:6]}\n-------------")

Number of Unique Encounters: 27891
Since this is the same as number of rows, this means no duplicates are found in the Encounter IDs 
-------------
Unique values of Encounters Class: ['ambulatory' 'outpatient' 'wellness' 'urgentcare' 'inpatient' 'emergency']
-------------
Example values of Diagnoses: ['Hyperlipidemia' 'Malignant tumor of colon'
 'Non-small cell lung cancer (disorder)'
 'Non-small cell carcinoma of lung  TNM stage 1 (disorder)'
 'Acute bronchitis (disorder)']
-------------


##### Organisation Table
Since this data set only has data from 1 hospital, this table doesn't serve much purpose

In [12]:
organizations.head(2)

,Id,NAME,ADDRESS,CITY,STATE,ZIP,LAT,LON
0,d78e84ec-30aa-3bba-a33a-f29a3a454662,MASSACHUSETTS GENERAL HOSPITAL,55 FRUIT STREET,BOSTON,MA,2114,42.362813,-71.069187


##### Patients Table
The original table contained patient names as well. However despite being synthetic data, to comply with Patient privacy (HIPPA compliance) and adhering to industry standards, the columns were dropped.

In [14]:
patients.head(2)

,Id,BIRTHDATE,YEAR_OF_BIRTH,DEATHDATE,MARITAL,RACE,ETHNICITY,GENDER,BIRTHPLACE,ADDRESS,CITY,STATE,COUNTY,ZIP,LAT,LON
0,5605b66b-e92d-c16c-1b83-b8bf7040d51f,19-03-1977,1977,NaN,M,white,nonhispanic,F,Wakefield Massachusetts US,510 Little Station Unit 69,Quincy,Massachusetts,Norfolk County,2186.0,42.290937,-70.975503
1,6e5ae27c-8038-7988-e2c0-25a103f01bfa,19-02-1940,1940,NaN,M,white,nonhispanic,M,Brookline Massachusetts US,747 Conn Throughway,Boston,Massachusetts,Suffolk County,2135.0,42.308831,-71.063162


In [15]:
print(f"Shape of the patient table: {patients.shape}")
print(f"Columns in the patient table: {patients.columns}\n-------------")

Shape of the patient table: (974, 16)
Columns in the patient table: Index(['Id', 'BIRTHDATE', 'YEAR_OF_BIRTH', 'DEATHDATE', 'MARITAL', 'RACE',
       'ETHNICITY', 'GENDER', 'BIRTHPLACE', 'ADDRESS', 'CITY', 'STATE',
       'COUNTY', 'ZIP', 'LAT', 'LON'],
      dtype='object')
-------------


In [16]:
# Basic Data Points to note (Patient Demographics)
print(f"Summary of Patient: {patients.RACE.value_counts()}\n-------------")
print(f"Summary of Patient: {patients.GENDER.value_counts()}\n-------------")
print(f"Summary of Patient: {patients.ETHNICITY.value_counts()}\n-------------")

Summary of Patient: RACE
white       680
black       163
asian        91
other        16
hawaiian     13
native       11
Name: count, dtype: int64
-------------
Summary of Patient: GENDER
M    494
F    480
Name: count, dtype: int64
-------------
Summary of Patient: ETHNICITY
nonhispanic    783
hispanic       191
Name: count, dtype: int64
-------------


##### Payers Table
Which was the insurance the patients were on? Can be used through the Payer ID In the encounter table

In [18]:
payers.head(2)

,Id,NAME,ADDRESS,CITY,STATE_HEADQUARTERED,ZIP,PHONE
0,b3221cfc-24fb-339e-823d-bc4136cbc4ed,Dual Eligible,7500 Security Blvd,Baltimore,MD,21244.0,1-877-267-2323
1,7caa7254-5050-3b5e-9eae-bd5ea30e809c,Medicare,7500 Security Blvd,Baltimore,MD,21244.0,1-800-633-4227


In [19]:
print(f"Shape of the payers table: {payers.shape}")
print("Columns in the payers table: ")
print(payers.columns)

Shape of the payers table: (10, 7)
Columns in the payers table: 
Index(['Id', 'NAME', 'ADDRESS', 'CITY', 'STATE_HEADQUARTERED', 'ZIP', 'PHONE'], dtype='object')


In [20]:
# Basic Data Points to note (Payer Details)
print(f"Payor Names: {payers.NAME.unique()}\n-------------")
print(f"Payor HQs: {payers.STATE_HEADQUARTERED.unique()}\n-------------")

Payor Names: ['Dual Eligible' 'Medicare' 'Medicaid' 'Humana' 'Blue Cross Blue Shield'
 'UnitedHealthcare' 'Aetna' 'Cigna Health' 'Anthem' 'NO_INSURANCE']
-------------
Payor HQs: ['MD' 'KY' 'IL' 'MN' 'CT' 'IN' nan]
-------------


##### Procedures Table
On each encounter, whether a procedure took place? This might be including MedProcs like Cancer Surgery, Biopsies, Tests and many more

In [22]:
procedures.head(2)

,START,STOP,PROC_YEAR,PROC_MONTH,PATIENT,ENCOUNTER,CODE,DESCRIPTION,BASE_COST,REASONCODE,REASONDESCRIPTION
0,2011-01-02,2011-01-02,2011,1,3de74169-7f67-9304-91d4-757e0f3a14d2,32c84703-2481-49cd-d571-3899d5820253,265764009.0,Renal dialysis (procedure),903,NaN,NaN
1,2011-01-03,2011-01-03,2011,1,d9ec2e44-32e9-9148-179a-1653348cc4e2,c98059da-320a-c0a6-fced-c8815f3e3f39,76601001.0,Intramuscular injection,2477,NaN,NaN


In [23]:
print(f"Shape of the procedures table: {procedures.shape}")
print(f"Columns in the procedures table: {procedures.columns}")

Shape of the procedures table: (47701, 11)
Columns in the procedures table: Index(['START', 'STOP', 'PROC_YEAR', 'PROC_MONTH', 'PATIENT', 'ENCOUNTER',
       'CODE', 'DESCRIPTION', 'BASE_COST', 'REASONCODE', 'REASONDESCRIPTION'],
      dtype='object')


In [24]:
# Basic Data Points to note (Proc Details)
print(f"Examples of Procedure Reason (Diagnosis): {procedures.REASONDESCRIPTION.unique()[1:6]}\n-------------")
print(f"Example values of Procedures performed: {procedures.DESCRIPTION.unique()[1:6]}\n-------------")

Examples of Procedure Reason (Diagnosis): ['Malignant tumor of colon' 'Suspected lung cancer (situation)'
 'Non-small cell carcinoma of lung  TNM stage 1 (disorder)'
 'Atrial Fibrillation' 'Normal pregnancy']
-------------
Example values of Procedures performed: ['Intramuscular injection'
 'Combined chemotherapy and radiation therapy (procedure)'
 'Diagnostic fiberoptic bronchoscopy (procedure)'
 'Digital examination of rectum'
 'Magnetic resonance imaging for measurement of brain volume (procedure)']
-------------


### Fill Rate Analysis (Data Sanity Checks)
After giving each table a look, we know that the data is clean and free of any loading errors (from the perspective of industry grade data lakes)
The columns are not out of orders, each parameter in every column makes sense, the Primary keys are a perfect match when performing joins.

A final sanity check we can perform is fill rates analysis

In [26]:
my_dict = {
    'patients': set(patients.columns),
    'procedures': set(procedures.columns),
    'encounters': set(encounters.columns),
    'payers': set(payers.columns)
}

In [27]:
#Print out all the Tables and Column Values

for key, value in my_dict.items():
    print(f"Key: {key}, Value: {value}")

Key: patients, Value: {'BIRTHDATE', 'ETHNICITY', 'Id', 'RACE', 'ZIP', 'YEAR_OF_BIRTH', 'LON', 'BIRTHPLACE', 'MARITAL', 'GENDER', 'STATE', 'COUNTY', 'DEATHDATE', 'CITY', 'ADDRESS', 'LAT'}
Key: procedures, Value: {'CODE', 'DESCRIPTION', 'PROC_MONTH', 'REASONCODE', 'PROC_YEAR', 'START', 'PATIENT', 'BASE_COST', 'STOP', 'ENCOUNTER', 'REASONDESCRIPTION'}
Key: encounters, Value: {'CODE', 'DESCRIPTION', 'CLAIM_YEAR', 'Id', 'ORGANIZATION', 'REASONCODE', 'PAYER', 'BASE_ENCOUNTER_COST', 'PAYER_COVERAGE', 'START', 'CLAIM_MONTH', 'PATIENT', 'ENCOUNTERCLASS', 'STOP', 'TOTAL_CLAIM_COST', 'REASONDESCRIPTION'}
Key: payers, Value: {'NAME', 'Id', 'PHONE', 'ZIP', 'STATE_HEADQUARTERED', 'CITY', 'ADDRESS'}


In [28]:
df_fill_rate = pd.DataFrame(columns=['Table', 'Field', 'Total Rows', 'Filled Rows', 'Fill Rate'])
# Calculate FIll rates seperately, and then make a final single table

# 1] Calculate Fill Rates for the patients table
for column_name in patients.columns:
    my_dict = {
        'Table': 'patients',
        'Field': f"{column_name}",
        'Total Rows': len(patients[f"{column_name}"]),
        'Filled Rows': patients[f"{column_name}"].count(),
        'Fill Rate': (patients[f"{column_name}"].count() / len(patients[f"{column_name}"])) * 100    # Example values
    }
    df_i = pd.DataFrame(columns=['Table', 'Column', 'Total Rows', 'Filled Rows', 'Fill Rate'])
    df_i = pd.DataFrame([my_dict])
    df_fill_rate = pd.concat([df_fill_rate, df_i], ignore_index=True)

# 2] Calculate Fill Rates for the procedures table
for column_name in procedures.columns:
    my_dict = {
        'Table': 'procedures',
        'Field': f"{column_name}",
        'Total Rows': len(procedures[f"{column_name}"]),
        'Filled Rows': procedures[f"{column_name}"].count(),
        'Fill Rate': (procedures[f"{column_name}"].count() / len(procedures[f"{column_name}"])) * 100
    }
    df_i = pd.DataFrame(columns=['Table', 'Column', 'Total Rows', 'Filled Rows', 'Fill Rate'])
    df_i = pd.DataFrame([my_dict])
    df_fill_rate = pd.concat([df_fill_rate, df_i], ignore_index=True)

# 3] Calculate Fill Rates for the payors table
for column_name in payers.columns:
    my_dict = {
        'Table': 'payers',
        'Field': f"{column_name}",
        'Total Rows': len(payers[f"{column_name}"]),
        'Filled Rows': payers[f"{column_name}"].count(),
        'Fill Rate': (payers[f"{column_name}"].count() / len(payers[f"{column_name}"])) * 100
    }
    df_i = pd.DataFrame(columns=['Table', 'Column', 'Total Rows', 'Filled Rows', 'Fill Rate'])
    df_i = pd.DataFrame([my_dict])
    df_fill_rate = pd.concat([df_fill_rate, df_i], ignore_index=True)

# 4] Calculate Fill Rates for the organizations table
for column_name in organizations.columns:
    my_dict = {
        'Table': 'organizations',
        'Field': f"{column_name}",
        'Total Rows': len(organizations[f"{column_name}"]),
        'Filled Rows': organizations[f"{column_name}"].count(),
        'Fill Rate': (organizations[f"{column_name}"].count() / len(organizations[f"{column_name}"])) * 100
    }
    df_i = pd.DataFrame(columns=['Table', 'Column', 'Total Rows', 'Filled Rows', 'Fill Rate'])
    df_i = pd.DataFrame([my_dict])
    df_fill_rate = pd.concat([df_fill_rate, df_i], ignore_index=True)

# 5] Calculate Fill Rates for the encounters table
for column_name in encounters.columns:
    my_dict = {
        'Table': 'encounters',
        'Field': f"{column_name}",
        'Total Rows': len(encounters[f"{column_name}"]),
        'Filled Rows': encounters[f"{column_name}"].count(),
        'Fill Rate': (encounters[f"{column_name}"].count() / len(encounters[f"{column_name}"])) * 100
    }
    df_i = pd.DataFrame(columns=['Table', 'Field', 'Total Rows', 'Filled Rows', 'Fill Rate'])
    df_i = pd.DataFrame([my_dict])
    df_fill_rate = pd.concat([df_fill_rate, df_i], ignore_index=True)

C:\Users\shant\AppData\Local\Temp\ipykernel_18068\3561849006.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_fill_rate = pd.concat([df_fill_rate, df_i], ignore_index=True)


In [29]:
# Merge Above result with Data Dictionary to get a final table
df_fill_rate_final = pd.merge(df_fill_rate, data_dict, on = ['Table', 'Field'], how='left')
df_fill_rate_final = df_fill_rate_final[['Table', 'Field', 'Total Rows', 'Filled Rows', 'Fill Rate', 'Description']]
df_fill_rate_final.to_csv('Fill_Rates.csv', index = False)
df_fill_rate_final

,Table,Field,Total Rows,Filled Rows,Fill Rate,Description
0,patients,Id,974,974,100.000000,Primary Key. Unique Identifier of the patient.
1,patients,BIRTHDATE,974,974,100.000000,The date (YYYY-MM-DD) the patient was born.
2,patients,YEAR_OF_BIRTH,974,974,100.000000,Patient's Year of Birth
3,patients,DEATHDATE,974,154,15.811088,The date (YYYY-MM-DD) the patient died.
4,patients,MARITAL,974,973,99.897331,"Marital Status. M is married, S is single. Cur..."
5,patients,RACE,974,974,100.000000,Description of the patient's primary race.
6,patients,ETHNICITY,974,974,100.000000,Description of the patient's primary ethnicity.
7,patients,GENDER,974,974,100.000000,"Gender. M is male, F is female."
8,patients,BIRTHPLACE,974,974,100.000000,Name of the town where the patient was born.
9,patients,ADDRESS,974,974,100.000000,Patient's street address without commas or new...


### Data Attribute Analysis
This section will explore tha actual numbers in this dataset, the various splits, the patient population, the demographic distribution, analysis of the diagnosis and procedures that the patient undergoes etc.

#### 1. Patient Attributes and Demographics: According to diagnosis types (Cancer vs Non Cancer)

Join the encounter table with the patients table; also manually assign each diagnosis type into cancer vs non cancer (external CSV for exact mapping)

In [32]:
diag_rollup_map = pd.read_csv("diagnosis_rollup.csv")
diag_rollup_map

,disease_name,rollup,rollup_category
0,Malignant tumor of colon,Colon Cancer,Cancer
1,Non-small cell lung cancer (disorder),Lung Cancer (NSCLC),Cancer
2,Non-small cell carcinoma of lung TNM stage 1 ...,Lung Cancer (NSCLC),Cancer
3,Primary small cell malignant neoplasm of lung ...,Lung Cancer (SCLC),Cancer
4,Malignant neoplasm of breast (disorder),Breast Cancer,Cancer
5,Primary malignant neoplasm of colon,Colon Cancer,Cancer
6,Overlapping malignant neoplasm of colon,Colon Cancer,Cancer
7,Neoplasm of prostate,Prostate Cancer,Cancer
8,Suspected lung cancer (situation),Lung Cancer,Cancer
9,Small cell carcinoma of lung (disorder),Lung Cancer (SCLC),Cancer


In [33]:
# Make a dataframe that contains the primary columns to be used
encounter_df = pd.merge(encounters, patients, left_on = 'PATIENT', right_on = 'Id', how = 'left')
encounter_df_final = pd.merge(encounter_df, diag_rollup_map, left_on = 'REASONDESCRIPTION', right_on = 'disease_name', how = 'left')
encounter_df_final = encounter_df_final[['Id_x', 'PATIENT', 'START', 'CLAIM_YEAR', 'CLAIM_MONTH', 'REASONDESCRIPTION', 'YEAR_OF_BIRTH', 'MARITAL', 'RACE', 'ETHNICITY', 'GENDER', 'rollup', 'rollup_category']]
encounter_df_final['AGE'] = encounter_df_final['CLAIM_YEAR'] - encounter_df_final['YEAR_OF_BIRTH']
encounter_df_final['rollup_category'] = encounter_df_final['rollup_category'].fillna('Unknown')
encounter_df_final['rollup'] = encounter_df_final['rollup'].fillna('Unknown')

# Let's bin the ages of the patients into various age groups
bins = [-float('inf'), 0, 18, 35, 50, 65, 80, 100, float('inf')]  # Adjust these bins as per your specific categories
# Define labels for each bin
labels = ['Invalid', '0-18', '19-35', '36-50', '51-65', '66-80', '80+', 'Invalid_']  # Adjust these labels accordingly

# Create a new column 'Age Category' based on the bins and display the final result
encounter_df_final['Age Group'] = pd.cut(encounter_df_final['AGE'], bins = bins, labels = labels, right = False)
encounter_df_final.head(2)

,Id_x,PATIENT,START,CLAIM_YEAR,CLAIM_MONTH,REASONDESCRIPTION,YEAR_OF_BIRTH,MARITAL,RACE,ETHNICITY,GENDER,rollup,rollup_category,AGE,Age Group
0,32c84703-2481-49cd-d571-3899d5820253,3de74169-7f67-9304-91d4-757e0f3a14d2,02-01-2011,2011,1,NaN,1928,M,white,nonhispanic,M,Unknown,Unknown,83,80+
1,c98059da-320a-c0a6-fced-c8815f3e3f39,d9ec2e44-32e9-9148-179a-1653348cc4e2,03-01-2011,2011,1,NaN,1964,M,white,nonhispanic,F,Unknown,Unknown,47,36-50


In [34]:
patient_demo = encounter_df_final.groupby(['MARITAL', 'RACE', 'ETHNICITY', 'GENDER', 'rollup_category', 'Age Group'])['PATIENT'].nunique().reset_index()
patient_demo.columns = ['Marital Status', 'Race', 'Ethnicity', 'Gender', 'Diagnosis Rolup', 'Age Group', 'Patient Count']
patient_demo = patient_demo[patient_demo['Patient Count'] > 0].reset_index(drop = True)
patient_demo.to_csv('Patient_Demographics.csv', index = False)
patient_demo.head(5)

C:\Users\shant\AppData\Local\Temp\ipykernel_18068\4219353192.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  patient_demo = encounter_df_final.groupby(['MARITAL', 'RACE', 'ETHNICITY', 'GENDER', 'rollup_category', 'Age Group'])['PATIENT'].nunique().reset_index()


,Marital Status,Race,Ethnicity,Gender,Diagnosis Rolup,Age Group,Patient Count
0,M,asian,hispanic,F,Non-Cancer,51-65,1
1,M,asian,hispanic,F,Non-Cancer,66-80,1
2,M,asian,hispanic,F,Non-Cancer,80+,2
3,M,asian,hispanic,F,Unknown,36-50,1
4,M,asian,hispanic,F,Unknown,51-65,1


#### 2. Longitudinal or Monthly Analysis of Patient Trends 

Analyse the encounter table on a monthly level, get monthly patient and claim counts for a longitudinal analysis and draw temporal insights on a diagnosis rollup level

In [36]:
longitudinal_clm_1 = encounter_df_final.groupby(['CLAIM_YEAR', 'CLAIM_MONTH', 'rollup', 'rollup_category'])['Id_x'].nunique().reset_index()
longitudinal_clm_1.columns = ['Claim Year', 'Claim Month', 'Diagnosis Rollup', 'Rollup Category', 'Claim Count']
longitudinal_clm_1 = longitudinal_clm_1[longitudinal_clm_1['Claim Count'] > 0].reset_index(drop = True)

longitudinal_clm_2 = encounter_df_final.groupby(['CLAIM_YEAR', 'CLAIM_MONTH'])['Id_x'].nunique().reset_index()
longitudinal_clm_2['rollup'] = 'Overall'
longitudinal_clm_2['rollup_category'] = 'Overall'
longitudinal_clm_2 = longitudinal_clm_2[['CLAIM_YEAR', 'CLAIM_MONTH', 'rollup', 'rollup_category', 'Id_x']]
longitudinal_clm_2.columns = ['Claim Year', 'Claim Month', 'Diagnosis Rollup', 'Rollup Category', 'Claim Count']
longitudinal_clm_2 = longitudinal_clm_2[longitudinal_clm_2['Claim Count'] > 0].reset_index(drop = True)
longitudinal_clm_2.head(5)

longitudinal_clm = pd.concat([longitudinal_clm_1, longitudinal_clm_2], ignore_index=True)
longitudinal_clm.to_csv('Claim_Monthly.csv', index = False)
longitudinal_clm.head(3)

,Claim Year,Claim Month,Diagnosis Rollup,Rollup Category,Claim Count
0,2011,1,Acute bronchitis (disorder),Non-Cancer,3
1,2011,1,Alzheimer's disease (disorder),Non-Cancer,3
2,2011,1,Colon Cancer,Cancer,1


In [37]:
longitudinal_ptn_1 = encounter_df_final.groupby(['CLAIM_YEAR', 'CLAIM_MONTH', 'rollup_category'])['PATIENT'].nunique().reset_index()
longitudinal_ptn_1.columns = ['Claim Year', 'Claim Month', 'Rollup Category', 'Patient Count']
longitudinal_ptn_1 = longitudinal_ptn_1[longitudinal_ptn_1['Patient Count'] > 0].reset_index(drop = True)
longitudinal_ptn_1.to_csv('Patient_Monthly.csv', index = False)

longitudinal_ptn_2 = encounter_df_final.groupby(['CLAIM_YEAR', 'CLAIM_MONTH'])['PATIENT'].nunique().reset_index()
longitudinal_ptn_2['rollup_category'] = 'Overall'
longitudinal_ptn_2 = longitudinal_ptn_2[['CLAIM_YEAR', 'CLAIM_MONTH', 'rollup_category', 'PATIENT']]
longitudinal_ptn_2.columns = ['Claim Year', 'Claim Month', 'Rollup Category', 'Patient Count']
longitudinal_ptn_2 = longitudinal_ptn_2[longitudinal_ptn_2['Patient Count'] > 0].reset_index(drop = True)

longitudinal_ptn = pd.concat([longitudinal_ptn_1, longitudinal_ptn_2], ignore_index=True)
longitudinal_ptn.to_csv('Patient_Monthly.csv', index = False)
longitudinal_ptn.head(3)

,Claim Year,Claim Month,Rollup Category,Patient Count
0,2011,1,Cancer,3
1,2011,1,Non-Cancer,16
2,2011,1,Unknown,52


### !!! IMPORTANT NOTE:

1. The decision to exclude the "Diagnosis Rollup" column from patient counts was made to prevent potential double-counting of patients across multiple Diagnosis rollup categories. Instead, a separate attribute labeled "Overall" was calculated to provide a unified count.

2. This approach ensures accuracy in calculations when the flat files are utilized in PowerBI, particularly when the tool aggregates data across different filters or slices.

3. However, this differentiation was not applied to Claim Counts, as claims do not double-count when aggregated across multiple attributes. Nonetheless, an attribute labeled "Overall" was still calculated for completeness.
____________________

#### 3. Payer Level Analysis

Explore the diversity of payers represented in the dataset to understand the range of insurance companies and payment options chosen by patients. This analysis provides insights into the financial aspects of healthcare coverage within the dataset.

In [40]:
payer_df = pd.merge(encounters, payers, left_on = 'PAYER', right_on = 'Id', how = 'left')
payer_df = pd.merge(payer_df, diag_rollup_map, left_on = 'REASONDESCRIPTION', right_on = 'disease_name', how = 'left')
payer_df = payer_df[['Id_x', 'PATIENT', 'PAYER', 'START', 'CLAIM_YEAR', 'CLAIM_MONTH', 'NAME','REASONDESCRIPTION', 'rollup', 'rollup_category']]
payer_df.head(3)

,Id_x,PATIENT,PAYER,START,CLAIM_YEAR,CLAIM_MONTH,NAME,REASONDESCRIPTION,rollup,rollup_category
0,32c84703-2481-49cd-d571-3899d5820253,3de74169-7f67-9304-91d4-757e0f3a14d2,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,02-01-2011,2011,1,NO_INSURANCE,NaN,NaN,NaN
1,c98059da-320a-c0a6-fced-c8815f3e3f39,d9ec2e44-32e9-9148-179a-1653348cc4e2,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,03-01-2011,2011,1,NO_INSURANCE,NaN,NaN,NaN
2,4ad28a3a-2479-782b-f29c-d5b3f41a001e,73babadf-5b2b-fee7-189e-6f41ff213e01,7caa7254-5050-3b5e-9eae-bd5ea30e809c,03-01-2011,2011,1,Medicare,NaN,NaN,NaN


In [41]:
payer_count = payer_df.groupby(['CLAIM_YEAR', 'CLAIM_MONTH', 'NAME', 'rollup_category'])['PATIENT'].nunique().reset_index()
payer_count.columns = ['Claim Year', 'Claim Month', 'Payer Name/Type', 'Diagnosis Rollup', 'Patient Count']
payer_count = payer_count[payer_count['Patient Count'] > 0].reset_index(drop = True)
payer_count.to_csv('Payer_Monthly_Counts.csv', index = False)
payer_count.head(3)

,Claim Year,Claim Month,Payer Name/Type,Diagnosis Rollup,Patient Count
0,2011,1,Aetna,Non-Cancer,2
1,2011,1,Anthem,Non-Cancer,2
2,2011,1,Dual Eligible,Non-Cancer,1


#### 4. Procedure Level Analysis
1. Types of Procedures: Identify the various types of procedures conducted at the hospital, categorizing them based on their medical nature and purpose.
2. Role of Pharmacy (Rx) Data: Pharmacy data is essential for tracking a patient's complete journey, including surgeries, adjuvant therapies, and neo-adjuvant therapies. However, this dataset lacks Pharmacy data, limiting the ability to follow holistic patient treatment paths.
3. Scope of Procedure Data: The dataset provides a record of procedures performed, but these procedures may not exclusively relate to cancer treatment or reflect a comprehensive patient journey.

In [43]:
# Read a manual mapping of procedure rollups (such as Biopsies, X Rays, surgeries, Chemotherapy etc.)
procedure_map = pd.read_csv("procedures_mapping.csv")
procedure_map

,procedure_description,procedure_rollup
0,Human epidermal growth factor receptor 2 gene ...,Biomarker Test
1,Human epidermal growth factor receptor 2 gene ...,Biomarker Test
2,Biopsy of prostate,Biopsy
3,Biopsy of breast (procedure),Biopsy
4,Biopsy of colon,Biopsy
5,Chemotherapy (procedure),Chemo
6,Combined chemotherapy and radiation therapy (p...,Chemo + Radiation
7,Colonoscopy,Colonoscopy
8,High resolution computed tomography of chest w...,CT Scan
9,Electrocardiographic procedure,ECG


In [44]:
procedure_df = pd.merge(procedures, procedure_map, left_on = 'DESCRIPTION', right_on = 'procedure_description', how = 'left')
procedure_df = pd.merge(procedure_df, diag_rollup_map, left_on = 'REASONDESCRIPTION', right_on = 'disease_name', how = 'left')
procedure_df = procedure_df[['PATIENT', 'ENCOUNTER', 'START', 'PROC_YEAR', 'PROC_MONTH', 'DESCRIPTION', 'REASONDESCRIPTION', 'procedure_rollup', 'rollup_category']]
procedure_df.head(3)

,PATIENT,ENCOUNTER,START,PROC_YEAR,PROC_MONTH,DESCRIPTION,REASONDESCRIPTION,procedure_rollup,rollup_category
0,3de74169-7f67-9304-91d4-757e0f3a14d2,32c84703-2481-49cd-d571-3899d5820253,2011-01-02,2011,1,Renal dialysis (procedure),NaN,Other Procedures,NaN
1,d9ec2e44-32e9-9148-179a-1653348cc4e2,c98059da-320a-c0a6-fced-c8815f3e3f39,2011-01-03,2011,1,Intramuscular injection,NaN,Other Procedures,NaN
2,d856d6e6-4c98-e7a2-129b-44076c63d008,2cfd4ddd-ad13-fe1e-528b-15051cea2ec3,2011-01-04,2011,1,Combined chemotherapy and radiation therapy (p...,Malignant tumor of colon,Chemo + Radiation,Cancer


In [45]:
procedure_count = procedure_df.groupby(['PROC_YEAR', 'PROC_MONTH', 'procedure_rollup', 'rollup_category'])['PATIENT'].nunique().reset_index()
procedure_count.columns = ['Procedure Year', 'Procedure Month', 'Procedure Rollup', 'Diagnosis Rollup', 'Patient Count']
procedure_count = procedure_count[procedure_count['Patient Count'] > 0].reset_index(drop = True)
procedure_count.to_csv('Procedure_Monthly_Counts.csv', index = False)
procedure_count.head(3)

,Procedure Year,Procedure Month,Procedure Rollup,Diagnosis Rollup,Patient Count
0,2011,1,Chemo + Radiation,Cancer,6
1,2011,1,MRI,Cancer,1
2,2011,1,Other Procedures,Cancer,1


This concludes the preliminary stage of the EDA. The flat files generated will be imported into PowerBI for making a report where further transformations might occur. Since the "business ask" is open ended the PowerBI report will focus on providing a complete look into the entire data structure in an intuitive easy to use manner. Following flat files will be used:
1. Fill_Rates.csv
2. Patient_Demographics.csv
3. Claim_Monthly.csv
4. Patient_Monthly.csv
5. Payer_Monthly_Counts.csv
6. Procedure_Monthly_Counts.csv

________________